# Advanced Querying Mongo

Importing libraries and setting up connection

In [3]:
from pymongo import MongoClient
import pandas as pd
import re


cursor=MongoClient()
cursor

db=cursor.companies
colec=db.companies

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [55]:
cursor.list_database_names()
data= pd.json_normalize(colec.find())
data.head(1).T

,0
_id,52cdef7c4bab8bd675297d8a
name,Wetpaint
permalink,abc2
crunchbase_url,http://www.crunchbase.com/company/wetpaint
homepage_url,http://wetpaint-inc.com
blog_url,http://digitalquarters.net/
blog_feed_url,http://digitalquarters.net/feed/
twitter_username,BachelrWetpaint
category_code,web
number_of_employees,47.0


In [32]:
query={'name': 'Babelgum'}
list(colec.find(query, {"name": 1}))
pd.json_normalize(query)

,name
0,Babelgum


### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [35]:
data = list(colec.find({'number_of_employees': {'$gte': 5000}},
                        {'name': 1, 'number_of_employees': 1}).sort('number_of_employees', -1).limit(20))
pd.json_normalize(data)

,_id,name,number_of_employees
0,52cdef7d4bab8bd67529941a,Siemens,405000
1,52cdef7c4bab8bd67529856a,IBM,388000
2,52cdef7d4bab8bd675299d33,Toyota,320000
3,52cdef7c4bab8bd675297e89,PayPal,300000
4,52cdef7e4bab8bd67529b0fe,Nippon Telegraph and Telephone Corporation,227000
5,52cdef7d4bab8bd675298aa4,Samsung Electronics,221726
6,52cdef7d4bab8bd675298b99,Accenture,205000
7,52cdef7e4bab8bd67529a657,Tata Consultancy Services,200300
8,52cdef7e4bab8bd67529aa51,Flextronics International,200000
9,52cdef7d4bab8bd675299156,Safeway,186000


### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [47]:
data1 = list(colec.find({'founded_year' : {'$gte' : 2000, '$lte' : 2005}},
                         {'name' : 1, 'founded_year' : 1}).limit(20))

pd.json_normalize(data1)

,_id,name,founded_year
0,52cdef7c4bab8bd675297d8a,Wetpaint,2005
1,52cdef7c4bab8bd675297d8c,Zoho,2005
2,52cdef7c4bab8bd675297d8d,Digg,2004
3,52cdef7c4bab8bd675297d8e,Facebook,2004
4,52cdef7c4bab8bd675297d8f,Omnidrive,2005
5,52cdef7c4bab8bd675297d95,StumbleUpon,2002
6,52cdef7c4bab8bd675297d96,Gizmoz,2003
7,52cdef7c4bab8bd675297d9a,Helio,2005
8,52cdef7c4bab8bd675297da1,Plaxo,2002
9,52cdef7c4bab8bd675297da5,Technorati,2002


### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [59]:
data2 = list(colec.find({'ipo.valuation_amount' : {'$gte' : 100000000},'founded_year' : {'$lte' : 2010}},
                         {'name' : 1, 'ipo' : 1}))
pd.json_normalize(data2)
                         

,_id,name,ipo.valuation_amount,ipo.valuation_currency_code,ipo.pub_year,ipo.pub_month,ipo.pub_day,ipo.stock_symbol
0,52cdef7c4bab8bd675297d8e,Facebook,104000000000,USD,2012.0,5.0,18.0,NASDAQ:FB
1,52cdef7c4bab8bd675297d94,Twitter,18100000000,USD,2013.0,11.0,7.0,NYSE:TWTR
2,52cdef7c4bab8bd675297de0,Yelp,1300000000,USD,2012.0,3.0,2.0,NYSE:YELP
3,52cdef7c4bab8bd675297e0c,LinkedIn,9310000000,USD,2011.0,7.0,20.0,NYSE:LNKD
4,52cdef7c4bab8bd675297e7a,Amazon,100000000000,USD,1997.0,5.0,NaN,NASDAQ:AMZN
5,52cdef7c4bab8bd675297e81,Brightcove,290000000,USD,2012.0,2.0,17.0,NASDAQ:BCOV
6,52cdef7c4bab8bd675297ee5,KIT digital,235000000,USD,2010.0,7.0,27.0,KITD
7,52cdef7c4bab8bd675297f15,Nielsen,1600000000,USD,2011.0,1.0,26.0,NYSE:NLSN
8,52cdef7c4bab8bd675297f36,OpenTable,1050000000,USD,2009.0,5.0,21.0,NASDAQ:OPEN
9,52cdef7c4bab8bd675297f42,ChannelAdvisor,287000000,USD,2013.0,5.0,23.0,NYSE:ECOM


### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [63]:
dat3 = list(colec.find({'number_of_employees' : {'$lt' : 1000}, 'founded_year' : {'$lt' : 2005}},
                       {'name': 1, 'number_of_employees': 1}).sort('number_of_employees', -1).limit(10))

pd.json_normalize(dat3)

,_id,name,number_of_employees
0,52cdef7d4bab8bd675298933,Infinera Corporation,974
1,52cdef7e4bab8bd67529ac95,NorthPoint Communications Group,948
2,52cdef7f4bab8bd67529be17,888 Holdings,931
3,52cdef7c4bab8bd6752986a2,Forrester Research,903
4,52cdef7e4bab8bd67529af6d,SonicWALL,900
5,52cdef7e4bab8bd67529b21b,Webmetrics,900
6,52cdef7e4bab8bd67529b3bd,Cornerstone OnDemand,881
7,52cdef7c4bab8bd675297ff0,Mozilla,800
8,52cdef7c4bab8bd67529857e,Buongiorno,800
9,52cdef7c4bab8bd675297de0,Yelp,800


### 6. All the companies that don't include the `partners` field.

In [64]:
dat4 = list(colec.find({'partners': {'$exists': False}}, {'name': 1}))
pd.DataFrame(dat4)



""


### 7. All the companies that have a null type of value on the `category_code` field.

In [70]:
dat5 = list(colec.find({'category_code' : None}, {'name' : 1}))
pd.json_normalize(dat5)

,_id,name
0,52cdef7c4bab8bd6752980f6,Collective
1,52cdef7c4bab8bd675298225,Snimmer
2,52cdef7c4bab8bd675298226,KoolIM
3,52cdef7c4bab8bd675298261,Level9 Media
4,52cdef7c4bab8bd675298262,VidKing
...,...,...
2746,52cdef7f4bab8bd67529c6c8,Nellix
2747,52cdef7f4bab8bd67529c6de,Cantimer
2748,52cdef7f4bab8bd67529c6e8,cruisecritic
2749,52cdef7f4bab8bd67529c6e9,Coloroot


### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [77]:
dat6 = list(colec.find({'number_of_employees' : {'$gt' : 100, '$lt' : 1000}},
                       {'name': 1, 'number_of_employees': 1}))
pd.json_normalize(dat6)

,_id,name,number_of_employees
0,52cdef7c4bab8bd675297d8b,AdventNet,600
1,52cdef7c4bab8bd675297da7,AddThis,120
2,52cdef7c4bab8bd675297da8,OpenX,305
3,52cdef7c4bab8bd675297db5,LifeLock,644
4,52cdef7c4bab8bd675297dbb,Jajah,110
...,...,...,...
748,52cdef7f4bab8bd67529c6ac,Inventa Technologies,207
749,52cdef7f4bab8bd67529c6d1,UOL (Universo Online),500
750,52cdef7f4bab8bd67529c6e6,OfficialVirtualDJ,102
751,52cdef7f4bab8bd67529c6f1,Willdan Group,385


### 9. Order all the companies by their IPO price in a descending order.

In [89]:
query = {'ipo.valuation_amount' : {'$gt': 0}}
filtro = {'name' : True, '_id' : False}

pd.json_normalize(list(colec.find(query, filtro).sort('ipo.valuation_amount', -1)))

,name
0,GREE
1,Facebook
2,Amazon
3,Twitter
4,Groupon
...,...
56,Energy and Power Solutions
57,Vringo
58,GT Solar
59,DeNA


### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [91]:
query = {}
filtro = {"_id": 0, "name": 1, "number_of_employees": 1}

result = colec.find(query, filtro).sort("number_of_employees", -1).limit(10)


pd.json_normalize(list(result))



,name,number_of_employees
0,Siemens,405000
1,IBM,388000
2,Toyota,320000
3,PayPal,300000
4,Nippon Telegraph and Telephone Corporation,227000
5,Samsung Electronics,221726
6,Accenture,205000
7,Tata Consultancy Services,200300
8,Flextronics International,200000
9,Safeway,186000


### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [95]:
query = {'founded_month' : {'$gte' : 6.0}}
filtro = {"name": 1, "number_of_employees": 1, 'founded_month' : 1}

pd.json_normalize(colec.find(query, filtro).limit(1000))

,_id,name,number_of_employees,founded_month
0,52cdef7c4bab8bd675297d8a,Wetpaint,47.0,10
1,52cdef7c4bab8bd675297d8c,Zoho,1600.0,9
2,52cdef7c4bab8bd675297d8d,Digg,60.0,10
3,52cdef7c4bab8bd675297d8f,Omnidrive,NaN,11
4,52cdef7c4bab8bd675297d90,Postini,NaN,6
...,...,...,...,...
995,52cdef7d4bab8bd675298d27,Openfilm,35.0,11
996,52cdef7d4bab8bd675298d29,uCubd,4.0,9
997,52cdef7d4bab8bd675298d2e,MyGreat,3.0,7
998,52cdef7d4bab8bd675298d32,SquareClock,10.0,12


### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [98]:
query = {'founded_year' : {'$lte' : 2000}, 'acquisition.price_amount' : {'$gt' : 10000}}
filtro = {"name": 1, "acquisition.price_amount": 1, 'founded_year' : 1}

pd.json_normalize(colec.find(query, filtro).limit(1000))

,_id,name,founded_year,acquisition.price_amount
0,52cdef7c4bab8bd675297d90,Postini,1999,625000000
1,52cdef7c4bab8bd675297deb,SideStep,1999,180000000
2,52cdef7c4bab8bd675297e2c,Recipezaar,1999,25000000
3,52cdef7c4bab8bd675297e37,Cyworld,1999,7140000
4,52cdef7c4bab8bd675297e89,PayPal,1998,1500000000
...,...,...,...,...
265,52cdef7f4bab8bd67529c6bb,Universal Microwave,1998,23200000
266,52cdef7f4bab8bd67529c6c3,Advanced Control Components,1982,18780000
267,52cdef7f4bab8bd67529c6ee,EnterSys Group,2000,17500000
268,52cdef7f4bab8bd67529c6f2,Contemporary Computer Services,1974,15040000


### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [104]:
query = {'acquisitions.acquired_year' : {'$gt' : 2010}}
filtro = {"name": 1, "acquisition": 1}

pd.json_normalize(colec.find(query, filtro).sort("acquisition.price_amount", -1))

,_id,name,acquisition.price_amount,acquisition.price_currency_code,acquisition.term_code,acquisition.source_url,acquisition.source_description,acquisition.acquired_year,acquisition.acquired_month,acquisition.acquired_day,acquisition.acquiring_company.name,acquisition.acquiring_company.permalink,acquisition
0,52cdef7d4bab8bd675298b89,LSI,6.600000e+09,USD,cash,http://dealbook.nytimes.com/2013/12/16/avago-t...,Avago to Buy LSI for $6.6 Billion,2013.0,12.0,16.0,Avago Technologies,avago-technologies,NaN
1,52cdef7c4bab8bd675297f28,Sabre,4.300000e+09,USD,None,http://venturebeat.com/2006/12/12/sabre-holdin...,"Sabre Holdings, travel reservation co., sold t...",2006.0,12.0,NaN,Silver Lake Partners & Texas Pacific Group,silver-lake-partners-texas-pacific-group,NaN
2,52cdef7d4bab8bd67529910e,The Weather Channel,3.500000e+09,USD,None,http://www.paidcontent.org/entry/419-deal-done...,Deal Done: Weather Channel Sold To NBCU and Ba...,2008.0,7.0,7.0,NBC Universal,nbcuniversal,NaN
3,52cdef7c4bab8bd6752981d6,SuccessFactors,3.400000e+09,USD,None,http://techcrunch.com/2012/02/22/successfactor...,After Closing $3.4B Acquisition Of SuccessFact...,2012.0,2.0,22.0,SAP,sap,NaN
4,52cdef7e4bab8bd67529aba6,Qualcomm Atheros,3.100000e+09,USD,stock,http://www.sdbj.com/news/2011/may/24/qualcomm-...,Qualcomm Brings Atheros Into the Fold,2011.0,5.0,5.0,Qualcomm,qualcomm,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
660,52cdef7f4bab8bd67529c626,Evolve IP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
661,52cdef7f4bab8bd67529c631,ProQuest,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
662,52cdef7f4bab8bd67529c637,Lenovo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
663,52cdef7f4bab8bd67529c63c,Layered Technologies,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [107]:
query = {}
filtro = {"name": 1, "founded_year": 1}

pd.json_normalize(colec.find(query, filtro).sort('founded_year', -1))

,_id,name,founded_year
0,52cdef7c4bab8bd675297fec,Fixya,2013.0
1,52cdef7c4bab8bd67529801f,Wamba,2013.0
2,52cdef7c4bab8bd6752982d4,Advaliant,2013.0
3,52cdef7c4bab8bd67529830a,Fluc,2013.0
4,52cdef7d4bab8bd675298ea7,iBazar,2013.0
...,...,...,...
18796,52cdef7f4bab8bd67529c6ec,Embedster,NaN
18797,52cdef7f4bab8bd67529c6f1,Willdan Group,NaN
18798,52cdef7f4bab8bd67529c6f4,Geekdive,NaN
18799,52cdef7f4bab8bd67529c6f8,goBookmaker,NaN


### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [109]:
query = {'founded_day' : {'$gte' : 7}}
filtro = {"name": 1, "acquisition.price_amount": 1}

pd.json_normalize(colec.find(query, filtro).sort("acquisition.price_amount", -1).limit(10))

,_id,name,acquisition.price_amount
0,52cdef7d4bab8bd675298934,Siebel Systems,5850000000
1,52cdef7d4bab8bd675299a88,Yammer,1200000000
2,52cdef7d4bab8bd6752996fe,Yammer,1200000000
3,52cdef7d4bab8bd675299421,ngmoco,400000000
4,52cdef7d4bab8bd675298f8a,Audible,300000000
5,52cdef7d4bab8bd675298922,Adify,300000000
6,52cdef7c4bab8bd67529872c,Zong,240000000
7,52cdef7e4bab8bd67529bc72,Topsy Labs,200000000
8,52cdef7c4bab8bd675298238,BreakingPoint Systems,160000000
9,52cdef7c4bab8bd67529816c,Goodreads,150000000


### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [117]:
query = {'category_code': 'web', "number_of_employees": {'$gte': 4000}}
filtro = {"name": 1, "number_of_employees": 1}

pd.json_normalize(colec.find(query, filtro).sort("number_of_employees", -1).limit(10))

,_id,name,number_of_employees
0,52cdef7c4bab8bd6752982a8,Experian,15500
1,52cdef7c4bab8bd675297d9b,eBay,15000
2,52cdef7c4bab8bd675297da3,Yahoo!,13600
3,52cdef7c4bab8bd675297fcb,Rakuten,10000
4,52cdef7c4bab8bd67529834c,Los Angeles Times Media Group,10000
5,52cdef7d4bab8bd675299d5d,Groupon,10000
6,52cdef7c4bab8bd675297ea4,Webkinz,8657
7,52cdef7c4bab8bd675297e96,AOL,8000
8,52cdef7c4bab8bd67529822a,Expedia,4400


### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [119]:
query = {'acquisition.price_currency_code': 'EUR', "acquisition.price_amount": {'$gte': 10000000}}
filtro = {"name": 1, "acquisition.price_amount": 1}

pd.json_normalize(colec.find(query, filtro).limit(10))

,_id,name,acquisition.price_amount
0,52cdef7c4bab8bd675297f02,ZYB,31500000
1,52cdef7d4bab8bd675298bf3,Apertio,140000000
2,52cdef7d4bab8bd675298f47,Greenfield Online,40000000
3,52cdef7e4bab8bd67529a536,Webedia,70000000
4,52cdef7e4bab8bd67529a729,Wayfinder,24000000
5,52cdef7e4bab8bd67529b747,Tuenti Technologies,70000000
6,52cdef7f4bab8bd67529c0cf,BioMed Central,43400000


### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [121]:
query = {"acquisition.acquired_month": {'$lte': 3}}
filtro = {"name": 1, "acquisition": 1}

pd.json_normalize(colec.find(query, filtro).limit(10))

,_id,name,acquisition.price_amount,acquisition.price_currency_code,acquisition.term_code,acquisition.source_url,acquisition.source_description,acquisition.acquired_year,acquisition.acquired_month,acquisition.acquired_day,acquisition.acquiring_company.name,acquisition.acquiring_company.permalink
0,52cdef7c4bab8bd675297dab,Kyte,NaN,USD,None,http://techcrunch.com/2011/01/31/exclusive-kit...,"KIT digital Acquires KickApps, Kewego AND Kyte...",2011,1,31.0,KIT digital,kit-digital
1,52cdef7c4bab8bd675297db4,NetRatings,327000000.0,USD,cash,http://login.vnuemedia.com/hr/login/login_subs...,Nielsen buys rest of NetRatings,2007,2,NaN,Nielsen,nielsen
2,52cdef7c4bab8bd675297dbf,blogTV,NaN,USD,None,http://techcrunch.com/2013/03/13/younow-buys-b...,Live Social Video Network YouNow Acquires Stre...,2013,3,13.0,YouNow,younow
3,52cdef7c4bab8bd675297dc0,Livestream,NaN,USD,cash_and_stock,,company,2008,1,1.0,Adviqo,adviqo
4,52cdef7c4bab8bd675297dda,iContact,169000000.0,USD,cash_and_stock,http://techcrunch.com/2012/02/28/vocus-buys-em...,Vocus Buys Email Marketing Company iContact Fo...,2012,2,28.0,Vocus,vocus
5,52cdef7c4bab8bd675297de8,Coghead,NaN,USD,None,http://www.techcrunch.com/2009/02/19/sap-acqui...,SAP Acquires Cogheadâ€™s Technology As It Look...,2009,2,19.0,SAP,sap
6,52cdef7c4bab8bd675297dee,Dailymotion,168000000.0,USD,None,http://techcrunch.com/2013/02/22/orange-acquir...,Orange Acquired Dailymotion For $168 Million B...,2013,2,22.0,Orange,orange
7,52cdef7c4bab8bd675297df4,Netvibes,NaN,USD,None,http://www.marketwatch.com/story/dassault-syst...,Dassault Systemes Acquires Netvibes,2012,2,9.0,Dassault Systemes,dassault
8,52cdef7c4bab8bd675297df8,Flickr,NaN,USD,cash_and_stock,http://news.cnet.com/Yahoo-buys-photo-sharing-...,CNET,2005,3,NaN,Yahoo!,yahoo
9,52cdef7c4bab8bd675297df9,BabyCenter,NaN,USD,None,http://www.investor.jnj.com/releasedetail.cfm?...,Johnson & Johnson Acquires BabyCenter From eTo...,2001,3,2.0,Johnson & Johnson,johnson-johnson


# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
bad_index = df.deadpooled_year[df.deadpooled_year.isna()].index
df.drop(bad_index, axis=0, inplace=True)
df = df[df.deadpooled_year > 3]
df.head()
df[‘deadpooled’]= df.deadpooled_year - df.founded_year
df.head()
df[df.deadpooled > 3].head()